In [105]:
import pandas as pd
import numpy as np

In [106]:
df = pd.read_csv("C:\\MyWork\\MyLearning\\ML\\Files\\DataSet\\Sentiment.csv")

In [107]:
df = df[['text','sentiment']]

In [108]:
set(df['sentiment'])

{'Negative', 'Neutral', 'Positive'}

In [109]:
df = df[df['sentiment'] != "Neutral"]

In [110]:
import re

df['text'] = df['text'].apply(lambda x : x.lower())
df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]','',x))

df['text'] = df['text'].apply(lambda x : re.sub('rt',' ',x))


In [111]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import sequence
from keras.preprocessing.sequence import pad_sequences

max_features = 2000
tokenizer = Tokenizer(num_words=max_features,split =' ')
tokenizer.fit_on_texts(df['text'].values)

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [112]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D

model = Sequential()
model.add(Embedding(max_features,128,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(196,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))

In [113]:
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics = ['accuracy'])


In [125]:
# y = df['sentiment'].map({'Positive' :1,'Negative' :0})
Y = pd.get_dummies(df['sentiment']).values

In [126]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

In [127]:
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)

(7188, 28)
(3541, 28)
(7188, 2)


In [128]:
model.fit(X_train,y_train,epochs=7,batch_size=32,verbose=2)


Epoch 1/7
 - 36s - loss: 0.4434 - accuracy: 0.8125
Epoch 2/7
 - 43s - loss: 0.3223 - accuracy: 0.8638
Epoch 3/7
 - 39s - loss: 0.2855 - accuracy: 0.8798
Epoch 4/7
 - 32s - loss: 0.2531 - accuracy: 0.8964
Epoch 5/7
 - 31s - loss: 0.2225 - accuracy: 0.9101
Epoch 6/7
 - 31s - loss: 0.2049 - accuracy: 0.9156
Epoch 7/7
 - 33s - loss: 0.1863 - accuracy: 0.9240


In [132]:
twt = ['Meetings: Because none of us is as dumb as all of us.']

twt = tokenizer.texts_to_sequences(twt)

twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)

print(twt)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  206  633    6  156    5   55 1050   55   46    6  156]]


In [133]:
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

negative
